In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')
!pip install -U -r /projects/icesat2_boreal/dps/requirements_main.txt

  Using cached pygeos-0.12.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached importlib_resources-5.7.1-py3-none-any.whl (28 kB)
  Using cached contextily-1.2.0-py3-none-any.whl (16 kB)
  Using cached fsspec-2022.3.0-py3-none-any.whl (136 kB)
  Using cached s3fs-2022.3.0-py3-none-any.whl (26 kB)
  Using cached rio_tiler-2.1.4-py3-none-any.whl
  Using cached pystac_client-0.3.3-py3-none-any.whl (19 kB)
  Using cached pystac-1.4.0-py3-none-any.whl (137 kB)
  Using cached xyzservices-2022.4.0-py3-none-any.whl (36 kB)
  Using cached aiobotocore-2.2.0-py3-none-any.whl
  Using cached botocore-1.24.21-py3-none-any.whl (8.6 MB)
  Using cached boto3-1.22.12-py3-none-any.whl (132 kB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Using cached boto3-1.22.11-py3-none-any.whl (132 kB)
  Using cached boto3-1.22.10-py3-none-any.whl (132 kB)
  Using cached boto3-1.22.9-py3-none-any.whl (132 kB)
  Using cached boto3-1.22.8-py3-none-any.whl (132 kB)
  Using c

# Launch DPS for mapBoreal.R

In [2]:
import os
import geopandas
import pandas as pd
import json
import datetime
!pip install xmltodict
import xmltodict

def local_to_s3(url, user='lduncanson'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')
def local_to_https(url, user='lduncanson'):
    ''' A Function to convert local paths to https urls'''
    return url.replace('/projects/my-private-bucket', f'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/{user}')
def local_to_https_uswest2(url, user='lduncanson'):
    ''' A Function to convert local paths to https us-west-s urls'''
    return url.replace('/projects/my-private-bucket', f'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/{user}')

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2022/03/ATL08_filt_tindex_master.csv')

#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv')
atl08_filt_tindex_master =   pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv')
topo_tindex_master =         pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
landsat_tindex_master =      pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/HLS_tindex_master.csv')

# Convert al local_paths to s3
#.. for data produced by 'lduncanson' workspace
atl08_filt_tindex_master['https'] = [local_to_https_uswest2(local_path, user='lduncanson') for local_path in atl08_filt_tindex_master['local_path']]

#.. for data produced by 'nathanmthomas' workspace
for tindex_master in [topo_tindex_master, landsat_tindex_master]:
    tindex_master['https'] = [local_to_https_uswest2(local_path, user='nathanmthomas') for local_path in tindex_master['local_path']]

# Use the ATL08 filtered tindex master list to tell you which tiles you'll run

In [4]:

NORWAY_TILE_LIST = pd.read_csv('/projects/my-public-bucket/misc_files/norway_tiles.csv').layer.tolist()

#NORWAY_TILE_LIST = [42,199,200,223,224,386,387,11,49,111,112,131]

DELTA_TILE_LIST = [3459]

BONA_TILE_LIST  = [3364,3365]

HEALY_TILE_LIST = [3552]

INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST
#INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST
#DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST
#len(DPS_INPUT_TILE_NUM_LIST)

#ALASKA_TILE_LIST =  list(range(3268,3272+1))+\
#                    list(range(3361,3366+1))+\
#                    list(range(3454,3459+1))+\
#                    list(range(3549,3555+1))+\
#                    list(range(3643,3648+1))

INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST
print(len(INPUT_EXPERIMENT_TILE_NUM_LIST))
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

FULL_TILE_LIST = atl08_filt_tindex_master['tile_num']
DPS_INPUT_TILE_NUM_LIST = FULL_TILE_LIST

DPS_INPUT_TILE_NUM_LIST = [33269, 757, 1447, 1391, 639, 640, 726, 727, 373, 405]
print(len(DPS_INPUT_TILE_NUM_LIST))


127
10


In [5]:
# Check retrieval of s3 path with a tle_num
in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
print(in_atl08_https)
print(in_topo_https)
print(in_landsat_https)

https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/dps_output/run_tile_atl08_ubuntu/tile_atl08/2022/04/18/19/14/30/554621/atl08_005_30m_filt_topo_landsat_20220418_33269.csv
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/master/2022/03/21/14/10/58/360867/Copernicus_33269_covars_cog_topo_stack.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/master/2022/04/01/19/45/33/837008/HLS_33269_06-15_09-01_2019_2021.tif


## Get files for boreal biomass models & boreal wide sample


In [6]:
bio_models_https = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models.tar'

train_data_https = 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/boreal_train_data_v5.csv'

In [7]:
bio_models_https

'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models.tar'

## Test from Terminal

In [12]:
# put latest test data in a test folder (outside repo)
# Terminal:
                           
# file 1: /projects/testing/input/atl08_004_30m_filt_topo_landsat_20220208_0042.csv
# file 2: /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
# file 3: /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#file 4: /projects/my-private-bucket/boreal_train_data_v3.csv
#file 5: /projects/my-private-bucket/bio_models.tar
#file 6: 42
# file 7:

# ./run_boreal_biomass.sh file1 file2 file3 file4 file5 file6 file7
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220322_0042.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v3.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 42 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 3 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

# of tiles to run: 115
# [3585, 4108, 1037, 28176, 4112, 31762, 1050, 2075, 28702, 31263, 29223, 29739, 30252, 3117, 30763, 3120, 2614, 2618, 2619, 1615, 1105, 1617, 2142, 612, 2148, 2149, 2151, 2152, 2153, 1146, 1662, 1159, 1672, 2698, 34443, 26280, 2217, 2223, 2224, 33975, 1721, 3770, 3771, 1214, 1731, 1733, 26827, 2779, 3301, 3302, 27368, 745, 3307, 1273, 2301, 27908, 1798, 1799, 1800, 3861, 28440, 3865, 31513, 28963, 31014, 29481, 2859, 29996, 30508, 1334, 4411, 3393, 3395, 2375, 3399, 1865, 1866, 1867, 2378, 2379, 2380, 2381, 3400, 3951, 3952, 34674, 3954, 2942, 895, 2944, 1933, 1934, 1937, 1938, 2457, 3486, 34210, 931, 1448, 26554, 4034, 4035, 1999, 981, 3029, 3030, 2009, 2010, 2011, 27098, 991, 992, 993, 2533, 27638]

In [13]:
# test of tile 1927
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220418_1927.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_1927_covars_cog_topo_stack.tif 
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_1927_06-15_09-01_2019_2021.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v4.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 1927 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn /projects/my-public-bucket/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/shared-buckets/nathanmthomas/atl08_004_30m_filt_merge_neighbors_3885.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_3585_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_3585_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



## Run a DPS job


In [26]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_boreal_biomass_quick'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-16gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; set to low (2-5) to test, 30-50 for production
    # 30 yields run times ~4 hours
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
    
    
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv',
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 2,
                                    'ppside': 2,
                                    'minDOY':1,
                                    'maxDOY':365,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 50,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='master',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )

    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 10
DPS run #: 1	| tile num: 33269	| submit status: success	| job id: 7a65036b-c30f-4fc6-ac10-bcc6258d96da
DPS run #: 2	| tile num: 757	| submit status: success	| job id: de5a827b-8e45-47d4-a90f-e5dcbe9a9977
Current time:	202205121923
CPU times: user 134 ms, sys: 3.61 ms, total: 137 ms
Wall time: 2.66 s


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,7a65036b-c30f-4fc6-ac10-bcc6258d96da,1,33269,2022-05-12 19:23:15.697520,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,de5a827b-8e45-47d4-a90f-e5dcbe9a9977,2,757,2022-05-12 19:23:15.768833,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,a8445c0d-9b72-477a-bce1-44c9fed88b5a,3,1447,2022-05-12 19:23:15.892305,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,86fe5736-4326-4b23-8037-1ebfc0a26064,4,1391,2022-05-12 19:23:16.086890,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,1a0fb4cd-32dc-4454-8ee0-f85f2d729383,5,639,2022-05-12 19:23:16.321660,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,f5fa2ae4-ebf9-48ff-9a6f-872d3643eb8e,6,640,2022-05-12 19:23:17.302965,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,8111ec8c-1d06-45b4-bcde-f1be59cb2e2e,7,726,2022-05-12 19:23:17.404682,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,de89d9bb-c601-461b-a513-d3908b4ddb8c,8,727,2022-05-12 19:23:17.627404,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,f2af57c5-a6f2-4007-8187-36faa57ccd3a,9,373,2022-05-12 19:23:17.739470,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb
0,success,200,4f9c403b-9c4d-494c-b35b-1330cce1fdf9,10,405,2022-05-12 19:23:17.995442,19,run_boreal_biomass_quick_ubuntu,lduncanson,maap-dps-worker-16gb


## Assess DPS Results

In [ ]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_v5_submission_results_4581_202205092316.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

In [11]:
submit_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4581 entries, 0 to 4580
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        4581 non-null   int64 
 1   status            4581 non-null   object
 2   http_status_code  4581 non-null   int64 
 3   job_id            4579 non-null   object
 4   dps_num           4581 non-null   int64 
 5   tile_num          4581 non-null   int64 
 6   submit_time       4581 non-null   object
 7   dbs_job_hour      4581 non-null   int64 
 8   algo_id           4581 non-null   object
 9   user              4581 non-null   object
 10  worker_type       4581 non-null   object
dtypes: int64(5), object(6)
memory usage: 393.8+ KB


In [22]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Succeeded'].iloc[0].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            '5e4fd861-5d92-4e5d-b5b8-1d4e8e6dd463'),
                           ('wps:Output',
                            OrderedDict([('@id',
                                          'output-2022-05-06T21:13:23.330599'),
                                         ('wps:Data',
                                          ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/lduncanson/dps_output/run_boreal_biomass_v4_ubuntu/master/2022/05/06/21/13/23/330599',
                                           's3:

In [53]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[20].job_id).content)

OrderedDict([('wps:Result',
              OrderedDict([('@xmlns:ows', 'http://www.opengis.net/ows/2.0'),
                           ('@xmlns:schemaLocation',
                            'http://schemas.opengis.net/wps/2.0/wps.xsd'),
                           ('@xmlns:wps', 'http://www.opengis.net/wps/2.0'),
                           ('@xmlns:xsi',
                            'http://www.w3.org/2001/XMLSchema-instance'),
                           ('wps:JobID',
                            'a58a94c3-a4c6-4b4f-ae20-95734d258ab2'),
                           ('wps:Output',
                            OrderedDict([('@id', 'traceback'),
                                         ('wps:Data',
                                          'activate does not accept more than one argument:\n[\'/app/icesat2_boreal/dps/alg_3-4/run_boreal_biomass.sh\', \'s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv\', \'input/Copernicus_571_covars_cog_topo_stack.tif\', \'inpu

## After successful DPS run, create AGB tindex

In [29]:
## create AGB tindex 
month_dir_str = '05'
index_out_dir = os.path.join('/projects/my-private-bucket/dps_output/run_boreal_biomass_qiuck_ubuntu/master/2022', month_dir_str)
!python /projects/icesat2_boreal/lib/build_tindex_master.py -t 'AGB' -y 2022 -m $month_dir_str -o $index_out_dir

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:
MAAP version:		master
Type:		AGB
Year:		2022
Month:		['05']
Days:		1-31

Output dir:  /projects/my-private-bucket/dps_output/run_boreal_biomass_qiuck_ubuntu/master/2022/05
                                             s3_path  ...                                     file
0  s3://maap-ops-workspace/lduncanson/dps_output/...  ...   boreal_agb_202205121652382252_0757.tif
1  s3://maap-ops-workspace/lduncanson/dps_output/...  ...   boreal_agb_202205121652382461_0373.tif
2  s3://maap-ops-workspace/lduncanson/dps_output/...  ...   boreal_agb_202205121652382513_1447.tif
3  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202205121652382538_33269.tif
4  

In [73]:
TEST_DPS  = True

if TEST_DPS:
    
    if True:
        #!python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t AGB -y 2022 -m $month_dir_str -o $index_out_dir
        t = pd.read_csv(os.path.join(index_out_dir,'AGB_tindex_master.csv'))
        COMPLETED_TILES = t.tile_num.to_list()
        NEED_TILES = list(set(DPS_INPUT_TILE_NUM_LIST) - set(COMPLETED_TILES))

        print(NEED_TILES)
        DPS_INPUT_TILE_NUM_LIST = NEED_TILES
    
else:
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST
   

#DPS_INPUT_TILE_NUM_LIST = [248, 273, 272, 271, 324]
print(f"# of tiles to run: {len(DPS_INPUT_TILE_NUM_LIST)}\n", DPS_INPUT_TILE_NUM_LIST)


NameError: name 'DPS_INPUT_TILE_NUM_LIST' is not defined

## Get other lists just of missing tiles (various ways)

In [22]:
need_tindex_master = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/Need_AGB_tindex_master.csv')
print(len(need_tindex_master))

INPUT_TILE_NUM_LIST = need_tindex_master.tile_num.tolist()

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

print(len(INPUT_TILE_NUM_LIST))

INPUT_TILE_NUM_LIST_NEED = INPUT_TILE_NUM_LIST

1
1


In [9]:
weird_tiles = [4304, 4305, 4221, 4220, 1785, 1718, 1720, 1661, 1257, 1318, 1317]
werid_tiles = [1255, 1196, 949, 1062, 1063, 1005, 950, 1004]
INPUT_TILE_NUM_LIST = weird_tiles
len(INPUT_TILE_NUM_LIST)

11

In [10]:
# Get all boreal tiles
ATL08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.
select_needs = [3360,2994,3190,2840,3012,3014,3017,2932,1261,1263,1264,988,978,794, 380,378,411,821,861,
                812,765,764,1308,1302,1469,1406,2495,2883,2965,3321,3509,3510,3327,3335,2976,2906,2907,2894,2814,4253,4293,4403,4440,4408,4372,4477,3986]

# Remove bad tiles
boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
#print(boreal_tile_index.head())
tile_matches_select_needs = boreal_tile_index.merge(ATL08_filt_tindex_master[ATL08_filt_tindex_master['tile_num'].isin(select_needs)], how='right', on='tile_num')
print(len(tile_matches_select_needs))
select_needs_filt = print([t for t in tile_matches_select_needs.tile_num])

45
[978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]


In [32]:
#select_needs_filt = [978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]

print(len(select_needs_filt))

import numpy as np
tile_nums_missing_but_wont_run = np.setdiff1d(select_needs, select_needs_filt)
print(len(tile_nums_missing_but_wont_run))
print(tile_nums_missing_but_wont_run)

tile_nums_missing_periphery = np.setdiff1d(INPUT_TILE_NUM_LIST_NEED, select_needs_filt)
print(len(tile_nums_missing_periphery))
print(tile_nums_missing_periphery)
INPUT_TILE_NUM_LIST = tile_nums_missing_periphery
len(INPUT_TILE_NUM_LIST)    

45
4
[3986 4253 4293 4408]
70
[  65  106  127  250  361  417  493  566  568  618  741  753  786  798
  801  830  848  876  877  927 1273 1287 1318 1402 1438 1439 1440 1604
 1848 1850 1852 1936 2425 2426 2952 3124 3355 3540 3542 3634 3728 3823
 3845 3846 3895 3916 4004 4007 4018 4083 4176 4204 4250 4256 4307 4325
 4342 4395 4417 4444 4450 4451 4463 4467 4468 4483 4499 4510 4527 4534]


70

In [18]:
INPUT_TILE_NUM_LIST = [4440,4372,4477]

## Send output cog.tif tiles somewhere

In [12]:
import glob
out_files = glob.glob("/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/**/*cog.tif", recursive=True)

In [13]:
print(out_files)

['/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/02/23/49/35/272239/boreal_agb_20220202_0200_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/42/37/290043/boreal_agb_20220203_0224_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/52/46/430279/boreal_agb_20220203_0387_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/02/21/50/348844/boreal_agb_20220203_0386_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/01/42/237812/boreal_agb_20220203_0199_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/06/47/336472/boreal_agb_20220203_0223_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/08/52/15295

In [ ]:
TEST_SUBDIR = 'add_solar'
OUTDIR = f'/projects/my-private-bucket/dps_output/run_boreal_biomass_v2_ubuntu/master/2022/{TEST_SUBDIR}'
!python /projects/icesat2_boreal/lib/build_tindex_master.py -y 2022 -m $TEST_SUBDIR -o $OUTDIR